# Training the tf_cnn_model
This file is used to train the tf_cnn_model and save it as a .keras zip archive, which can be loaded and reused as required later on.  

The libraries cell are common between the two for ease of development.

In [6]:
#Loading Libraries

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random
import cv2
from glob import glob
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, classification_report
from pathlib import Path
from PIL import Image 

In [8]:
#Loading the data 

data = os.listdir(r"./input/Digits/Digits" )
data_X = []     
data_y = []  
num_classes = len(data)
for i in range (0,num_classes):
    data_list = os.listdir(r"./input/Digits/Digits" +"/"+str(i))
    for j in data_list:
        pic = cv2.imread(r"./input/Digits/Digits" +"/"+str(i)+"/"+j)
        pic = cv2.resize(pic,(32,32))
        data_X.append(pic)
        data_y.append(i)

         
if len(data_X) == len(data_y) :
    print("Total Data points =",len(data_X))

# Labels and images
data_X = np.array(data_X)
data_y = np.array(data_y)

Total Data points = 10160


# SPLITTING DATASET

Splitting the dataset into test, train and validation sets. Preprocessing for the features (images) into grayscale, enhancing it with histogram equalization and then normalizing. Followed by converting then into a NumPy array. further reshaping the image's array and using data augmentation. Preprocessing for the labels involves one-hot encoding the label classes

In [9]:
#Spliting the train validation and test sets

train_X, test_X, train_y, test_y = train_test_split(data_X,data_y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)
print("Training Set Shape = ",train_X.shape)
print("Validation Set Shape = ",valid_X.shape)
print("Test Set Shape = ",test_X.shape)

Training Set Shape =  (7721, 32, 32, 3)
Validation Set Shape =  (1931, 32, 32, 3)
Test Set Shape =  (508, 32, 32, 3)


In [10]:
# Preprocessing the images for neuralnet

def PreprocessImage(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #making image grayscale
    img = cv2.equalizeHist(img) #Histogram equalization to enhance contrast
    img = img/255 #normalizing
    return img

train_X = np.array(list(map(PreprocessImage, train_X)))
test_X = np.array(list(map(PreprocessImage, test_X)))
valid_X= np.array(list(map(PreprocessImage, valid_X)))

#Reshaping the images
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)

#Augmentation
image_data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
image_data_generator.fit(train_X)

In [11]:
# One hot encoding of the labels

train_y = to_categorical(train_y, num_classes)
test_y = to_categorical(test_y, num_classes)
valid_y = to_categorical(valid_y, num_classes)

# MODEL BUILDING

**For this Model, we will build a convolutional neural network.**
* Initialising the Convnet
* Defining by adding layers
* Compiling the Convnet
* Train the Convnet

In [12]:
#Creating a Neural Network

tf_cnn_model = Sequential()

tf_cnn_model.add((Conv2D(60,(5,5),input_shape=(32, 32, 1) ,padding = 'Same' ,activation='relu')))
tf_cnn_model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
tf_cnn_model.add(MaxPooling2D(pool_size=(2,2)))
#tf_cnn_model.add(Dropout(0.25))

tf_cnn_model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
tf_cnn_model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
tf_cnn_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
tf_cnn_model.add(Dropout(0.5))

tf_cnn_model.add(Flatten())
tf_cnn_model.add(Dense(500,activation='relu'))
tf_cnn_model.add(Dropout(0.5))
tf_cnn_model.add(Dense(10, activation='softmax'))

tf_cnn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 60)        1560      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 60)        90060     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 60)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 30)        16230     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 30)        8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 30)         0         
 2D)                                                    

2024-03-14 00:09:15.039571: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [14]:
#Compiling the model

rms_Optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon = 1e-08)
tf_cnn_model.compile(optimizer=rms_Optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#Fit the model

historical_data = tf_cnn_model.fit(image_data_generator.flow(train_X, train_y, batch_size=32),
                              epochs = 30, validation_data = (valid_X, valid_y),
                              verbose = 2, steps_per_epoch= 200)

Epoch 1/30


2024-03-14 00:12:34.996858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


200/200 - 56s - loss: 0.9485 - accuracy: 0.6737 - val_loss: 0.1125 - val_accuracy: 0.9591 - 56s/epoch - 281ms/step
Epoch 2/30
200/200 - 56s - loss: 0.2728 - accuracy: 0.9138 - val_loss: 0.0596 - val_accuracy: 0.9839 - 56s/epoch - 278ms/step
Epoch 3/30
200/200 - 54s - loss: 0.1877 - accuracy: 0.9401 - val_loss: 0.0571 - val_accuracy: 0.9834 - 54s/epoch - 270ms/step
Epoch 4/30
200/200 - 65s - loss: 0.1461 - accuracy: 0.9556 - val_loss: 0.0333 - val_accuracy: 0.9881 - 65s/epoch - 327ms/step
Epoch 5/30
200/200 - 66s - loss: 0.1295 - accuracy: 0.9597 - val_loss: 0.0525 - val_accuracy: 0.9839 - 66s/epoch - 329ms/step
Epoch 6/30
200/200 - 63s - loss: 0.1065 - accuracy: 0.9688 - val_loss: 0.0326 - val_accuracy: 0.9917 - 63s/epoch - 317ms/step
Epoch 7/30
200/200 - 63s - loss: 0.1095 - accuracy: 0.9668 - val_loss: 0.0857 - val_accuracy: 0.9777 - 63s/epoch - 313ms/step
Epoch 8/30
200/200 - 72s - loss: 0.1025 - accuracy: 0.9691 - val_loss: 0.0343 - val_accuracy: 0.9902 - 72s/epoch - 358ms/step
Epo

In [15]:
tf_cnn_model.save('./output/model.keras')

In [16]:
# Testing the tf_cnn_model on the test set

score = tf_cnn_model.evaluate(test_X, test_y, verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

Test Score =  0.010815313085913658
Test Accuracy = 0.998031497001648
